In [2]:
import numpy as np
import scipy
import scipy.io as sio
from scipy.sparse import csr_matrix
from scipy.optimize import least_squares

In [3]:
def obj_fun_linear(w, C, out, n0, A, X):
    out[0:n0] = np.maximum(np.zeros([n0, 1]),out[0:n0])
    obj = np.sum(np.multiply(C, np.multiply(out, out))) / 2.0 + np.dot(np.transpose(w), w) / 2.0
    grad = w - np.transpose(np.transpose(np.multiply(C, out)) * A * X)
    sv = out[0:n0] > 0, abs(out[n0:]) > 0
    return obj[0, 0], grad, sv

In [4]:
def hess_vect_mult(w, sv, C, grad, n0, A, X):
    w = np.transpose(np.matrix(w))
    y = w
    z = np.multiply(np.multiply(C, sv), A * (X * w))
    y = y + np.transpose((np.transpose(z) * A) * X) + grad
    y = y.A1
    return y

In [5]:
def line_search_linear(w, d, out, C, n0 ,A, X):
    t = 0
    Xd = A * (X * d)

    while 1:
        out2 = out - t * Xd
        sv = np.nonzero( scipy.vstack(( out2[0:n0] > 0, abs(out2[n0:]) > 0 )) )[0]
        g = np.transpose(w) * d + t * np.transpose(d) * d - np.transpose(np.multiply(C[sv], out2[sv])) * Xd[sv]
        h = np.transpose(d) * d + np.transpose(Xd[sv]) * np.multiply(Xd[sv], C[sv])
        g, h = g[0, 0], h[0, 0]
        t = t - g / h
        if g * g / h < 1e-8: 
            break
    out = out2
    return t, out

In [6]:
def rank_svm(X_, S_, O_):

    # opt
    max_itr = 10
    cg_max_itr = 20
    prec = 1e-8

    X = X_; A = O_; B = S_;
    n0, d = A.shape[0], X.shape[1]
    w = scipy.matrix(scipy.zeros([d, 1]))

    itr = 0
    C = scipy.matrix(0.1*scipy.ones([A.shape[0]+B.shape[0], 1]))

    out1 = scipy.matrix(scipy.vstack( (scipy.ones([A.shape[0], 1]), scipy.zeros([B.shape[0], 1])) ))
    A = scipy.sparse.vstack((A, B))
    out = out1 - A * X * w

    while 1:
        itr += 1
        if itr > max_itr:
            print("Maximum number of Newton steps reached")
            break


        obj, grad, sv = obj_fun_linear(w, C, out, n0, A, X)
        sv = scipy.vstack(sv)
        res = least_squares(hess_vect_mult, np.zeros(w.shape[0]), ftol = 1e-8, xtol = 1e-8, gtol = 1e-8, args = (sv, C, grad, n0, A, X))
        step = np.transpose(np.matrix(res.x))
        t, out = line_search_linear(w, step, out, C, n0, A, X)
        w += t * step;

        check = - np.transpose(step) * grad
        check = check[0, 0]
        print(check, prec*obj)
        if check < prec * obj: 
            break

        print("Iteration = ", itr)

    print(w)
    return w

In [7]:
datadict = sio.loadmat('./relative_attributes_v2/relative_attributes/pubfig/data.mat')

In [8]:
attr_names = []
for x in datadict['attribute_names'][0]:
    attr_names.append(x[0])
datadict['attribute_names'] = attr_names

datadict['im_names'] = datadict['im_names'][0]
datadict['class_labels'] = datadict['class_labels'][:, 0]
datadict['used_for_training'] = datadict['used_for_training'][:, 0]
datadict['class_names'] = datadict['class_names'][0]

In [8]:
WeightMatrix=[]
for p in range(len(datadict['attribute_names'])):
    cat_ordering = datadict['relative_ordering'][p]
    Sdata, Odata = [[],[],[]], [[],[],[]]
    num = datadict['feat'].shape[0]
    for i in range(len(datadict['class_labels'])):
        im1_lab = datadict['class_labels'][i] - 1
        for j in range(len(datadict['class_labels'][i+1:])):
            im2_lab = datadict['class_labels'][i+1:][j] - 1
            if cat_ordering[im1_lab] == cat_ordering[im2_lab]:
                Sdata[0].append(len(Sdata[0])/2)
                Sdata[0].append(len(Sdata[0])/2)
                Sdata[1].append(i)
                Sdata[1].append(i+j+1)
                Sdata[2].append(-1)
                Sdata[2].append(1)

            elif cat_ordering[im1_lab] < cat_ordering[im2_lab]:
                Odata[0].append(len(Odata[0])/2)
                Odata[0].append(len(Odata[0])/2)
                Odata[1].append(i)
                Odata[1].append(i+j+1)
                Odata[2].append(-1)
                Odata[2].append(1)
            elif cat_ordering[im1_lab] > cat_ordering[im2_lab]:
                Odata[0].append(len(Odata[0])/2)
                Odata[0].append(len(Odata[0])/2)
                Odata[1].append(i)
                Odata[1].append(i+j+1)
                Odata[2].append(1)
                Odata[2].append(-1)

    S = csr_matrix((Sdata[2], (Sdata[0], Sdata[1])),(int(len(Sdata[0])/2), num))
    O = csr_matrix((Odata[2], (Odata[0], Odata[1])),(int(len(Odata[0])/2), num))
    X = scipy.matrix(datadict['feat'])
    w = rank_svm(X, S, O)
    WeightMatrix.append(w)


MemoryError: 

In [13]:
#indicates smiling attribute
#order is 
# Male
# White
# Young
# Smiling
# Chubby
# VisibleForehead
# BushyEyebrows
# NarrowEyes
# PointyNose
# BigLips
# RoundFace
# 3 indicates smiling
print(np.dot(X[423],WeightMatrix[0]))

[[-1.64102452]]


In [16]:
np.save('./results/WeightMatrixosr.npy',WeightMatrix)

In [21]:
w=np.load('./results/WeightMatrix.npy')

In [29]:
for i,x in enumerate(datadict['im_names']):      
    if x[0]=='ScarlettJohansson_331.jpg':  
        print(i)
        break

541


In [22]:
X=datadict['feat']
print(np.dot(X[659].T,w[0]))

ValueError: shapes (512,) and (542,1) not aligned: 512 (dim 0) != 542 (dim 0)

In [56]:
minSmile=500
name="";
for x in range(len(datadict['im_names'])):
    if np.dot(X[x],w1[3]) < minSmile:
        minSmile=np.dot(X[x],w1[3])
        name=datadict['im_names'][x]
name

array(['ViggoMortensen_179.jpg'], dtype='<U22')

In [61]:
maxChubby=-500
name="";
for x in range(len(datadict['im_names'])):
    if np.dot(X[x],w1[4]) > maxChubby:
        maxChubby=np.dot(X[x],w1[4])
        name=datadict['im_names'][x]
name

array(['AlexRodriguez_133.jpg'], dtype='<U21')

In [32]:
datadict['class_labels'][-1]

8